In [1]:
import pandas as pd
user = pd.read_json('user.json',lines=True)
business = pd.read_json('business.json',lines=True)
review = pd.read_json('review.json',lines=True, chunksize=10000)

### review 只採計 2018 年以後的評論

In [2]:
date_threshold = pd.Timestamp('2018-01-01')
review_df = pd.DataFrame([], columns = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'])
for chunk in review:
    new_chunk = chunk[(chunk['date'] > date_threshold)]
    review_df = pd.concat([review_df, new_chunk], ignore_index=True)
# from IPython.display import display
# # 秀出前兩個 chunks
# date_threshold = pd.Timestamp('2018-01-01')
# for _, df_partial in zip(range(2), review):
#     display(df_partial[(df_partial['date'] > date_threshold)])

In [35]:
user_list = review_df['user_id'].unique().tolist()
business_list = review_df['business_id'].unique().tolist()

### 地區選定為 las vegas、category 包含 restaurants & bars、有營業、在 2018 年後有被評論

In [36]:
city_mask = business['city'] == 'Las Vegas'
cat_res_mask = business['categories'].str.contains("Restaurants")
cat_bar_mask = business['categories'].str.contains("Bars")
review_2018_mask = business['business_id'].isin(business_list)
is_open_mask = business['is_open'] == 1
target_business_df = business[city_mask & cat_res_mask & cat_bar_mask & review_2018_mask & is_open_mask]

### 在 2018 年有 elite 資格

In [37]:
target_user_df = user[(user['elite'].str.contains("2018"))]

### 篩選後資料概況

In [38]:
res_mask = review_df['business_id'].isin(target_business_df['business_id'])
user_mask = review_df['user_id'].isin(target_user_df['user_id'])
target_review_df = review_df[res_mask & user_mask]

In [39]:
print('評論數：',len(target_review_df))
print('評論人數：',len(target_review_df['user_id'].unique()))
print('餐廳數：',len(target_review_df['business_id'].unique()))

評論數： 10588
評論人數： 4499
餐廳數： 759


In [ ]:
target_review_df.to_csv('2018_lasvegas_bar_elite_review.csv', index=False)